In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

# plt.style.use('seaborn-colorblind')
# %matplotlib inline
#from feature_cleaning import rare_values as ra

## Load Dataset

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)


In [3]:
data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,1,53.1000
4,0,3,male,35.0,0,8.0500
5,0,3,male,NaN,0,8.4583
6,0,1,male,54.0,0,51.8625
7,0,3,male,2.0,3,21.0750
8,1,3,female,27.0,0,11.1333
9,1,2,female,14.0,1,30.0708


In [4]:
# Note that we include target variable in the X_train 
# because we need it to supervise our discretization
# this is not the standard way of using train-test-split
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

## Normalization - Standardization (Z-score scaling)

removes the mean and scales the data to unit variance.<br />z = (X - X.mean) /  std

In [5]:
# add the new created feature
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_zscore'] = ss.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_zscore
857         1       1    male  51.0      0  26.5500    -0.122530
52          1       1  female  49.0      1  76.7292     0.918124
386         0       3    male   1.0      5  46.9000     0.299503
124         0       1    male  54.0      0  77.2875     0.929702
578         0       3  female   NaN      1  14.4583    -0.373297
549         1       2    male   8.0      1  36.7500     0.089005


In [6]:
# check if it is with mean=0 std=1
print(X_train_copy['Fare_zscore'].mean())
print(X_train_copy['Fare_zscore'].std())


5.417460665907666e-17
1.000803535686101


## Min-Max scaling
transforms features by scaling each feature to a given range. Default to [0,1].<br />X_scaled = (X - X.min / (X.max - X.min)

In [6]:
# add the new created feature
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_minmax'] = mms.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_minmax
857         1       1    male  51.0      0  26.5500     0.051822
52          1       1  female  49.0      1  76.7292     0.149765
386         0       3    male   1.0      5  46.9000     0.091543
124         0       1    male  54.0      0  77.2875     0.150855
578         0       3  female   NaN      1  14.4583     0.028221
549         1       2    male   8.0      1  36.7500     0.071731


In [7]:
# check the range of Fare_minmax
print(X_train_copy['Fare_minmax'].max())
print(X_train_copy['Fare_minmax'].min())

1.0
0.0


In [9]:
from feature_cleaning import outlier as ot
index,para = ot.outlier_detect_IQR(data=X_train_copy,col='Fare_minmax',threshold=7)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 13
Proportion of outlier detected 0.02086677367576244
Upper bound: 0.3818345704285448 
Lower bound: -0.305101680716227


In [18]:
index,para = ot.outlier_detect_arbitrary(data=X_train_copy,col='Fare_minmax',upper_fence=0.99,lower_fence=0.0001)
print('Upper bound:',para[0],'\nLower bound:',para[1])

Num of outlier detected: 9
Proportion of outlier detected 0.014446227929373997
Upper bound: 0.99 
Lower bound: 0.0001


## Robust scaling
removes the median and scales the data according to the quantile range (defaults to IQR)<br />X_scaled = (X - X.median) / IQR

In [9]:
# add the new created feature
from sklearn.preprocessing import RobustScaler
rs = RobustScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_robust'] = rs.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Survived  Pclass     Sex   Age  SibSp     Fare  Fare_robust
857         1       1    male  51.0      0  26.5500     0.492275
52          1       1  female  49.0      1  76.7292     2.630973
386         0       3    male   1.0      5  46.9000     1.359616
124         0       1    male  54.0      0  77.2875     2.654768
578         0       3  female   NaN      1  14.4583    -0.023088
549         1       2    male   8.0      1  36.7500     0.927011
